# Problem 1

### Part one

In [5]:
import pandas as pd

# File path to the image
file_path = '/Users/juanjo/Library/CloudStorage/OneDrive-UniversityofOklahoma/2024 Fall/Systems Optimization/Final Exam/FinalExam2024_Data.xlsx'


# Load the Excel file into a DataFrame
df = pd.read_excel(file_path)

# Convert columns to lists
items = df['ITEM'].tolist()
weights = df['Weight (pounds)'].tolist()
values = df['Value'].tolist()

# Display the lists
print("Items:", items)
print("Weights:", weights)
print("Values:", values)

Items: ['Item 1', 'Item 2', 'Item 3', 'Item 4', 'Item 5', 'Item 6', 'Item 7', 'Item 8', 'Item 9', 'Item 10', 'Item 11', 'Item 12', 'Item 13', 'Item 14', 'Item 15', 'Item 16', 'Item 17', 'Item 18', 'Item 19', 'Item 20', 'Item 21', 'Item 22', 'Item 23', 'Item 24', 'Item 25', 'Item 26', 'Item 27', 'Item 28', 'Item 29', 'Item 30']
Weights: [4, 5, 5, 9, 9, 4, 7, 7, 6, 3, 7, 8, 3, 8, 4, 3, 7, 9, 6, 7, 6, 8, 7, 9, 3, 4, 5, 4, 3, 8]
Values: [3, 2, 5, 4, 4, 4, 4, 9, 7, 9, 8, 9, 8, 3, 6, 5, 7, 8, 2, 2, 7, 2, 5, 3, 7, 4, 7, 10, 4, 6]


In [148]:
from gurobipy import *
import pandas as pd

# File path to the image
file_path = '/Users/juanjo/Library/CloudStorage/OneDrive-UniversityofOklahoma/2024 Fall/Systems Optimization/Final Exam/FinalExam2024_Data.xlsx'


# Load the Excel file into a DataFrame
df = pd.read_excel(file_path)

# Convert columns to lists
items = df['ITEM'].tolist()
weights = df['Weight (pounds)'].tolist()
values = df['Value'].tolist()

'''# Display the lists
print("Items:", items)
print("Weights:", weights)
print("Values:", values)'''

m = Model("knapsack")

m.setParam('OutputFlag', 0)

x = m.addVars((item for item in items), vtype = GRB.BINARY, name="x")

allowed_weight = 15

# Add weight constraint
m.addConstr(
    quicksum(x[item] * weights[i] for i, item in enumerate(items)) <= allowed_weight,
    name="weight_constraint"
)

# Set the objective to maximize total profit
m.setObjective(quicksum(values[i] * x[item] for i, item in enumerate(items)), GRB.MAXIMIZE)

m.optimize() 

# Print the solution
if m.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for i, item in enumerate(items):
        if x[item].x > 0.5:  # If the variable value is close to 1, the item is selected
            print(f"{item}: Included (Weight = {weights[i]}, Value = {values[i]})")
    print(f"Total Value: {m.objVal}")
else:
    print("No optimal solution found.")


Optimal solution found:
Item 10: Included (Weight = 3, Value = 9)
Item 13: Included (Weight = 3, Value = 8)
Item 25: Included (Weight = 3, Value = 7)
Item 28: Included (Weight = 4, Value = 10)
Total Value: 34.0


### Part Two

In [150]:
# Create a Gurobi model
m = Model("knapsack_two_bags")


m.setParam('OutputFlag', 0)

# Define weight capacities for the two bags
weight_bag1 = 15
weight_bag2 = 50


# Add binary decision variables for each item in both bags
x_bag1 = m.addVars(items, vtype=GRB.BINARY, name="x_bag1")
x_bag2 = m.addVars(items, vtype=GRB.BINARY, name="x_bag2")

# Add weight constraints for each bag
m.addConstr(
    quicksum(x_bag1[item] * weights[i] for i, item in enumerate(items)) <= weight_bag1,
    name="weight_constraint_bag1"
)
m.addConstr(
    quicksum(x_bag2[item] * weights[i] for i, item in enumerate(items)) <= weight_bag2,
    name="weight_constraint_bag2"
)

# Ensure that each item is assigned to at most one bag
m.addConstrs(
    (x_bag1[item] + x_bag2[item] <= 1 for item in items),
    name="assignment_constraint"
)

# Set the objective to maximize the total value of items in both bags
m.setObjective(
    quicksum(values[i] * (x_bag1[item] + x_bag2[item]) for i, item in enumerate(items)),
    GRB.MAXIMIZE
)

# Optimize the model
m.optimize()

# Print the solution
if m.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    total_weight_bag1 = 0
    total_weight_bag2 = 0
    total_value = m.objVal

    for i, item in enumerate(items):
        if x_bag1[item].x > 0.5:  # If the item is selected for Bag 1
            print(f"{item}: Included in Bag 1 (Weight = {weights[i]}, Value = {values[i]})")
            total_weight_bag1 += weights[i]  # Add the weight to Bag 1 total weight
        elif x_bag2[item].x > 0.5:  # If the item is selected for Bag 2
            print(f"{item}: Included in Bag 2 (Weight = {weights[i]}, Value = {values[i]})")
            total_weight_bag2 += weights[i]  # Add the weight to Bag 2 total weight

    # Print total weight and value for each bag
    print(f"Total Weight in Bag 1: {total_weight_bag1}")
    print(f"Total Weight in Bag 2: {total_weight_bag2}")
    print(f"Total Value: {total_value}")
else:
    print("No optimal solution found.")

Optimal solution found:
Item 3: Included in Bag 2 (Weight = 5, Value = 5)
Item 6: Included in Bag 2 (Weight = 4, Value = 4)
Item 8: Included in Bag 2 (Weight = 7, Value = 9)
Item 9: Included in Bag 2 (Weight = 6, Value = 7)
Item 10: Included in Bag 2 (Weight = 3, Value = 9)
Item 11: Included in Bag 2 (Weight = 7, Value = 8)
Item 12: Included in Bag 2 (Weight = 8, Value = 9)
Item 13: Included in Bag 2 (Weight = 3, Value = 8)
Item 15: Included in Bag 2 (Weight = 4, Value = 6)
Item 16: Included in Bag 2 (Weight = 3, Value = 5)
Item 25: Included in Bag 1 (Weight = 3, Value = 7)
Item 27: Included in Bag 1 (Weight = 5, Value = 7)
Item 28: Included in Bag 1 (Weight = 4, Value = 10)
Item 29: Included in Bag 1 (Weight = 3, Value = 4)
Total Weight in Bag 1: 15
Total Weight in Bag 2: 50
Total Value: 98.0


# Problem 2

### Part 1

In [152]:
from gurobipy import Model, GRB, quicksum

# Define the data
cities = [1, 2, 3, 4, 5, 6, 7]  # City labels
n = len(cities)

# Travel costs and times for airplane (matrix form)
plane_cost = [
    [0, 85, 160, 99, 280, 194, 302],
    [123, 0, 223, 232, 385, 234, 312],
    [162, 240, 0, 202, 175, 213, 209],
    [150, 248, 197, 0, 291, 295, 349],
    [344, 327, 193, 314, 0, 258, 132],
    [202, 176, 201, 281, 246, 0, 181],
    [326, 271, 230, 318, 130, 174, 0],
]
plane_time = [
    [0, 0.9, 1.1, 1, 2.8, 1.8, 2.5],
    [0.8, 0, 1.7, 1.8, 3.1, 1.8, 2.4],
    [1.1, 1.6, 0, 1.4, 1.7, 1.7, 1.7],
    [0.9, 1.9, 1.3, 0, 2.7, 2.8, 2.9],
    [2.8, 3, 1.6, 2.8, 0, 2, 1.3],
    [1.9, 1.7, 1.8, 2.8, 2.2, 0, 1.2],
    [2.6, 2.5, 1.7, 2.9, 1.3, 1.2, 0],
]


# Create the Gurobi model
model = Model("TSP_planes_only")

model.setParam('OutputFlag', 0)

# Decision variables
x_plane = model.addVars(n, n, vtype=GRB.BINARY, name="x_plane")  # Binary for plane routes
u = model.addVars(n, vtype=GRB.CONTINUOUS, lb=1, ub=n, name="u")  # Position variables for subtour elimination

# Objective: Minimize total travel time
model.setObjective(
    quicksum(plane_time[i][j] * x_plane[i, j] for i in range(n) for j in range(n) if i != j),
    GRB.MINIMIZE
)

# Constraints

# 1. Each city must be visited exactly once
for i in range(n):
    model.addConstr(
        quicksum(x_plane[i, j] for j in range(n) if j != i) == 1,
        name=f"visit_once_out_{i}"
    )
    model.addConstr(
        quicksum(x_plane[j, i] for j in range(n) if j != i) == 1,
        name=f"visit_once_in_{i}"
    )

# 2. Subtour elimination (MTZ constraints)
for i in range(1, n):
    for j in range(1, n):
        if i != j:
            model.addConstr(
                u[i] - u[j] + n * (x_plane[i, j]) <= n - 1,
                name=f"subtour_elimination_{i}_{j}"
            )

# 3. Fix City 7 (hometown) as the starting and ending point
hometown = 6
model.addConstr(
    quicksum(x_plane[hometown, j] for j in range(n) if j != hometown) == 1,
    name="start_at_hometown"
)
model.addConstr(
    quicksum(x_plane[i, hometown] for i in range(n) if i != hometown) == 1,
    name="end_at_hometown"
)



# Optimize the model
model.optimize()

# Print the solution
if model.status == GRB.OPTIMAL:
    total_cost = sum(
    plane_cost[i][j] * x_plane[i, j].x for i in range(n) for j in range(n) if i != j
    )
    print("\nOptimal solution found:")
    print(f"Total time (hours): {model.objVal}")
    print(f"Total Cost (USD): {total_cost:.2f}")
    print("Routes:")
    for i in range(n):
        for j in range(n):
            if i != j and x_plane[i, j].x > 0.5:
                    print(f"Plane: {cities[i]} -> {cities[j]}")

else:
    print("No optimal solution found.")


Optimal solution found:
Total time (hours): 9.0
Total Cost (USD): 1076.00
Routes:
Plane: 1 -> 4
Plane: 2 -> 1
Plane: 3 -> 5
Plane: 4 -> 3
Plane: 5 -> 7
Plane: 6 -> 2
Plane: 7 -> 6


### Part 2

In [154]:
from gurobipy import Model, GRB, quicksum

# Define the data
cities = [1, 2, 3, 4, 5, 6, 7]  # City labels
n = len(cities)

# Travel costs and times for bus (matrix form)
bus_cost = [
    [0, 21, 21, 25, 38, 27, 25],
    [13, 0, 30, 32, 31, 17, 25],
    [24, 16, 0, 12, 19, 32, 21],
    [25, 24, 19, 0, 21, 36, 38],
    [22, 42, 16, 39, 0, 17, 11],
    [23, 27, 16, 34, 25, 0, 13],
    [19, 19, 25, 37, 28, 19, 0],
]
bus_time = [
    [0, 4.7, 5.4, 4.5, 10.1, 7, 9.4],
    [4.5, 0, 7.1, 6.7, 10.8, 6.9, 10.2],
    [3.5, 6.1, 0, 5.1, 6.4, 7.4, 7.6],
    [2.8, 6.6, 5.9, 0, 10.7, 10.7, 10.7],
    [9.3, 11.4, 7.6, 10.5, 0, 7.4, 5],
    [6.9, 7, 7.6, 9.5, 9, 0, 5.4],
    [8.9, 9.3, 6.7, 11.3, 6.2, 4.2, 0],
]


# Create the Gurobi model
model = Model("TSP_planes_only")

model.setParam('OutputFlag', 0)

# Decision variables
x_bus = model.addVars(n, n, vtype=GRB.BINARY, name="x_plane")  # Binary for bus routes
u = model.addVars(n, vtype=GRB.CONTINUOUS, lb=1, ub=n, name="u")  # Position variables for subtour elimination

# Objective: Minimize total travel cost
model.setObjective(
    quicksum(bus_cost[i][j] * x_bus[i, j] for i in range(n) for j in range(n) if i != j),
    GRB.MINIMIZE
)

# Constraints

# 1. Each city must be visited exactly once
for i in range(n):
    model.addConstr(
        quicksum(x_bus[i, j] for j in range(n) if j != i) == 1,
        name=f"visit_once_out_{i}"
    )
    model.addConstr(
        quicksum(x_bus[j, i] for j in range(n) if j != i) == 1,
        name=f"visit_once_in_{i}"
    )

# 2. Subtour elimination (MTZ constraints)
for i in range(1, n):
    for j in range(1, n):
        if i != j:
            model.addConstr(
                u[i] - u[j] + n * (x_bus[i, j]) <= n - 1,
                name=f"subtour_elimination_{i}_{j}"
            )

# 3. Fix City 7 (hometown) as the starting and ending point
hometown = 6
model.addConstr(
    quicksum(x_bus[hometown, j] for j in range(n) if j != hometown) == 1,
    name="start_at_hometown"
)
model.addConstr(
    quicksum(x_bus[i, hometown] for i in range(n) if i != hometown) == 1,
    name="end_at_hometown"
)



# Optimize the model
model.optimize()

# Print the solution
if model.status == GRB.OPTIMAL:
    total_time = sum(
        bus_time[i][j] * x_bus[i, j].x for i in range(n) for j in range(n) if i != j
    )
    print("\nOptimal solution found:")
    print(f"Total travel cost: {model.objVal}")
    print(f"Total travel time: {total_time:.2f} hours")
    print("Routes:")
    for i in range(n):
        for j in range(n):
            if i != j and x_bus[i, j].x > 0.5:
                print(f"Bus: {cities[i]} -> {cities[j]}")
else:
    print("No optimal solution found.")


Optimal solution found:
Total travel cost: 116.0
Total travel time: 47.80 hours
Routes:
Bus: 1 -> 3
Bus: 2 -> 1
Bus: 3 -> 4
Bus: 4 -> 5
Bus: 5 -> 6
Bus: 6 -> 7
Bus: 7 -> 2


### Part 3

In [156]:
from gurobipy import Model, GRB, quicksum

# Define the data
cities = [1, 2, 3, 4, 5, 6, 7]  # City labels
n = len(cities)

# Travel costs and times for bus (matrix form)
bus_cost = [
    [0, 21, 21, 25, 38, 27, 25],
    [13, 0, 30, 32, 31, 17, 25],
    [24, 16, 0, 12, 19, 32, 21],
    [25, 24, 19, 0, 21, 36, 38],
    [22, 42, 16, 39, 0, 17, 11],
    [23, 27, 16, 34, 25, 0, 13],
    [19, 19, 25, 37, 28, 19, 0],
]
bus_time = [
    [0, 4.7, 5.4, 4.5, 10.1, 7, 9.4],
    [4.5, 0, 7.1, 6.7, 10.8, 6.9, 10.2],
    [3.5, 6.1, 0, 5.1, 6.4, 7.4, 7.6],
    [2.8, 6.6, 5.9, 0, 10.7, 10.7, 10.7],
    [9.3, 11.4, 7.6, 10.5, 0, 7.4, 5],
    [6.9, 7, 7.6, 9.5, 9, 0, 5.4],
    [8.9, 9.3, 6.7, 11.3, 6.2, 4.2, 0],
]

# Travel costs and times for airplane (matrix form)
plane_cost = [
    [0, 85, 160, 99, 280, 194, 302],
    [123, 0, 223, 232, 385, 234, 312],
    [162, 240, 0, 202, 175, 213, 209],
    [150, 248, 197, 0, 291, 295, 349],
    [344, 327, 193, 314, 0, 258, 132],
    [202, 176, 201, 281, 246, 0, 181],
    [326, 271, 230, 318, 130, 174, 0],
]
plane_time = [
    [0, 0.9, 1.1, 1, 2.8, 1.8, 2.5],
    [0.8, 0, 1.7, 1.8, 3.1, 1.8, 2.4],
    [1.1, 1.6, 0, 1.4, 1.7, 1.7, 1.7],
    [0.9, 1.9, 1.3, 0, 2.7, 2.8, 2.9],
    [2.8, 3, 1.6, 2.8, 0, 2, 1.3],
    [1.9, 1.7, 1.8, 2.8, 2.2, 0, 1.2],
    [2.6, 2.5, 1.7, 2.9, 1.3, 1.2, 0],
]
# Total time constraint
T_max = 20

# Create the Gurobi model
model = Model("TSP_with_Modes")

model.setParam('OutputFlag', 0)

# Decision variables
x_bus = model.addVars(n, n, vtype=GRB.BINARY, name="x_bus")  # Binary for bus routes
x_plane = model.addVars(n, n, vtype=GRB.BINARY, name="x_plane")  # Binary for plane routes
u = model.addVars(n, vtype=GRB.CONTINUOUS, lb=1, ub=n, name="u")  # Position variables for subtour elimination

# Objective: Minimize total travel cost
model.setObjective(
    quicksum(bus_cost[i][j] * x_bus[i, j] for i in range(n) for j in range(n) if i != j) +
    quicksum(plane_cost[i][j] * x_plane[i, j] for i in range(n) for j in range(n) if i != j),
    GRB.MINIMIZE
)

# Constraints

# 1. Each city must be visited exactly once
for i in range(n):
    model.addConstr(
        quicksum(x_bus[i, j] + x_plane[i, j] for j in range(n) if j != i) == 1,
        name=f"visit_once_out_{i}"
    )
    model.addConstr(
        quicksum(x_bus[j, i] + x_plane[j, i] for j in range(n) if j != i) == 1,
        name=f"visit_once_in_{i}"
    )

# 2. Subtour elimination (MTZ constraints)
for i in range(1, n):
    for j in range(1, n):
        if i != j:
            model.addConstr(
                u[i] - u[j] + n * (x_bus[i, j] + x_plane[i, j]) <= n - 1,
                name=f"subtour_elimination_{i}_{j}"
            )

# 3. Only one mode of transportation per route
for i in range(n):
    for j in range(n):
        if i != j:
            model.addConstr(x_bus[i, j] + x_plane[i, j] <= 1, name=f"one_mode_{i}_{j}")

# 4. Total travel time constraint

model.addConstr(
    quicksum(bus_time[i][j] * x_bus[i, j] for i in range(n) for j in range(n) if i != j) +
    quicksum(plane_time[i][j] * x_plane[i, j] for i in range(n) for j in range(n) if i != j) <= T_max,
    name="time_constraint"
)


# 5. Fix City 7 (hometown) as the starting and ending point
hometown = 6  # City 7 is indexed as 6
model.addConstr(
    quicksum(x_bus[hometown, j] + x_plane[hometown, j] for j in range(n) if j != hometown) == 1,
    name="start_at_hometown"
)
model.addConstr(
    quicksum(x_bus[i, hometown] + x_plane[i, hometown] for i in range(n) if i != hometown) == 1,
    name="end_at_hometown"
)

# Optimize the model
model.optimize()

# Print the solution
if model.status == GRB.OPTIMAL:
    total_time = sum(
        bus_time[i][j] * x_bus[i, j].x + plane_time[i][j] * x_plane[i, j].x
        for i in range(n) for j in range(n) if i != j
    )
    print("\nOptimal solution found:")
    print(f"Total travel cost: {model.objVal:.2f} USD")
    print(f"Total travel time: {total_time:.2f} hours")
    print("Routes:")
    for i in range(n):
        for j in range(n):
            if i != j and (x_bus[i, j].x > 0.5 or x_plane[i, j].x > 0.5):
                mode = "Bus" if x_bus[i, j].x > 0.5 else "Plane"
                print(f"{mode}: {cities[i]} -> {cities[j]}")
else:
    print("No optimal solution found.")


Optimal solution found:
Total travel cost: 643.00 USD
Total travel time: 19.80 hours
Routes:
Plane: 1 -> 2
Bus: 2 -> 6
Bus: 3 -> 4
Bus: 4 -> 1
Plane: 5 -> 3
Plane: 6 -> 7
Plane: 7 -> 5


# Problem 3

In [131]:
from gurobipy import Model, GRB, quicksum

# Data
plants = ["P1", "P2"]
cities = ["C1", "C2", "C3"]

cost = {
    ("P1", "C1"): 18, ("P1", "C2"): 17, ("P1", "C3"): 21,
    ("P2", "C1"): 13, ("P2", "C2"): 15, ("P2", "C3"): 10,
}

arc_capacity = {
    ("P1", "C1"): 15, ("P1", "C2"): 30, ("P1", "C3"): 20,
    ("P2", "C1"): 20, ("P2", "C2"): 25, ("P2", "C3"): 30,
}

demand = {"C1": 25, "C2": 15, "C3": 40}

# Model
model = Model("ProductionCapacityAndDistribution")

model.setParam('OutputFlag', 0)

# Decision variables
x = model.addVars(plants, cities, vtype=GRB.CONTINUOUS, name="x")  # Amount transported
y = model.addVars(plants, vtype=GRB.CONTINUOUS, name="y")  # Plant production capacity

# Objective: Minimize total cost
model.setObjective(
    quicksum(cost[i, j] * x[i, j] for i in plants for j in cities),
    GRB.MINIMIZE
)

# Constraints

# 1. Demand constraints
for j in cities:
    model.addConstr(
        quicksum(x[i, j] for i in plants) == demand[j],
        name=f"demand_{j}"
    )

# 2. Production capacity constraints
for i in plants:
    model.addConstr(
        quicksum(x[i, j] for j in cities) <= y[i],
        name=f"production_capacity_{i}"
    )

# 3. Arc capacity constraints
for i in plants:
    for j in cities:
        model.addConstr(
            x[i, j] <= arc_capacity[i, j],
            name=f"arc_capacity_{i}_{j}"
        )

# Optimize
model.optimize()

# Print solution
if model.status == GRB.OPTIMAL:
    print("\nOptimal solution found:")
    print(f"Total cost: {model.objVal:.2f}")
    for i in plants:
        print(f"{i} production capacity: {y[i].x:.2f} tons")
    for i in plants:
        for j in cities:
            if x[i, j].x > 0:
                print(f"Transport {x[i, j].x:.2f} tons from {i} to {j}")
else:
    print("No optimal solution found.")


Optimal solution found:
Total cost: 1085.00
P1 production capacity: 15.00 tons
P2 production capacity: 65.00 tons
Transport 5.00 tons from P1 to C1
Transport 10.00 tons from P1 to C3
Transport 20.00 tons from P2 to C1
Transport 15.00 tons from P2 to C2
Transport 30.00 tons from P2 to C3
